In [ ]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import math
import random
import matplotlib.pyplot as plt
import numpy as np
import tkinter as tk

In [ ]:
# Define the Agent class for Trees
class TreeAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.on_fire = False

    def step(self):
        if self.on_fire:
            # print("fire step")
            # Get the current location of the tree
            x, y = self.pos

            # Define adjacent cell offsets (assuming 4 neighbors)
            neighbors = [(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)]

             # If wind is high, include neighbors that are two cells away
            if self.model.wind == "high":
                neighbors.extend([(x + 2, y), (x - 2, y), (x, y + 2), (x, y - 2)])


            # Shuffle the list of neighbors randomly
            random.shuffle(neighbors)
            # print("neighbours of fire")
            # print(neighbors)

            # Ignite up to two random adjacent cells
            ignited = 0
            for neighbor_x, neighbor_y in neighbors:
                if 0 <= neighbor_x < self.model.grid.width and 0 <= neighbor_y < self.model.grid.height:
                    if ignited < 1 and not self.model.grid.is_cell_empty((neighbor_x, neighbor_y)):
                        cell_contents = self.model.grid.get_cell_list_contents((neighbor_x, neighbor_y))
                        for agent in cell_contents:
                            if isinstance(agent, TreeAgent):
                                self.model.ignite_tree(neighbor_x, neighbor_y)
                                print("fire spread")
                                ignited += 1
            # print("end fire step")

# Define the Agent class for Firefighters
class FirefighterAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.target_tree = None

    def step(self):
        # print("firefighter step")

        # Find the nearest tree on fire
        nearest_fire_tree = self.find_nearest_fire_tree()

        if nearest_fire_tree:
            # Move to the nearest tree on fire
            # print(self.pos)
            # print(nearest_fire_tree.pos)
            current_position = self.pos
            delta_x = nearest_fire_tree.pos[0] - self.pos[0]
            delta_y = nearest_fire_tree.pos[1] - self.pos[1]
            
            if delta_x < 0:
                current_position = (current_position[0] - 1, current_position[1])
            elif delta_x > 0:
                current_position = (current_position[0] + 1, current_position[1])
            elif delta_y < 0:
                current_position = (current_position[0], current_position[1] - 1)
            elif delta_y > 0:
                current_position = (current_position[0], current_position[1] + 1)
            
            
            self.move_to_tree(current_position)

            # Extinguish the fire and remove the tree from the grid
            if (nearest_fire_tree.pos == current_position):
                print('Gets extinguished')
                self.extinguish_and_remove_tree(nearest_fire_tree)

    def find_nearest_fire_tree(self):
        fire_trees = []

        # Iterate over all cells in the grid
        for x in range(self.model.grid.width):
            for y in range(self.model.grid.height):
                cell_contents = self.model.grid.get_cell_list_contents([(x, y)])
                for agent in cell_contents:
                    if isinstance(agent, TreeAgent) and agent.on_fire:
                        fire_trees.append(agent)

        if fire_trees:
            # Calculate distances to all fire trees
            distances = [self.distance_to_tree(tree) for tree in fire_trees]

            # Find the nearest fire tree
            nearest_index = distances.index(min(distances))
            return fire_trees[nearest_index]

        return None
    

    def distance_to_tree(self, tree):
        return manhattan_distance(self.pos, tree.pos)

    def move_to_tree(self, pos):
        # Move to the location of the target tree
        self.model.grid.move_agent(self, pos)
    
    def extinguish_and_remove_tree(self, tree):
        # Extinguish the fire in the tree
        tree.on_fire = False

        # Remove the tree from the grid and the scheduler
        self.model.grid.remove_agent(tree)
        self.model.schedule.remove(tree)
        
# Define a class for Firebreak
class FirebreakAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.type = "Firebreak"  # Identify the agent type as Firebreak

    def step(self):
        pass  # FireBreak agents do not perform any actions
        
                
# Inside the BushfireModel class
class BushfireModel(Model):
    def __init__(self, width, height, tree_density, num_firefighters, wind="none"):
        self.num_agents = width * height
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)
        self.wind = wind

        # Create and place Tree agents based on tree_density
        for i in range(width):
            for j in range(height):
                if random.random() < tree_density:
                    tree = TreeAgent(self.random.randint(1, 1e6), self)
                    self.grid.place_agent(tree, (i, j))
                    self.schedule.add(tree)

        # # Create and place Firefighter agents
        # for i in range(num_firefighters):
        #     x = random.randrange(self.grid.width)
        #     y = random.randrange(self.grid.height)
        #     firefighter = FirefighterAgent(self.random.randint(1, 1e6), self)
        #     self.grid.place_agent(firefighter, (x, y))
        #     self.schedule.add(firefighter)

        # Define data collectors for model output
        self.datacollector = DataCollector(agent_reporters={"On Fire": lambda a: a.on_fire if isinstance(a, TreeAgent) else None})
        
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

    # Method to ignite a tree at a specified location
    def ignite_tree(self, x, y):
        cell_contents = self.grid.get_cell_list_contents([(x, y)])
        for agent in cell_contents:
            if isinstance(agent, TreeAgent):
                agent.on_fire = True
                # print("ignited fire")
    
    # Method to extinguish a fire at a specified location
    def extinguish_tree_fire(self, x, y):
        cell_contents = self.grid.get_cell_list_contents([(x, y)])
        for agent in cell_contents:
            if isinstance(agent, TreeAgent) and agent.on_fire:
                agent.on_fire = False
                

    # Define a function to add FirebreakAgents to the model
    def add_firebreak(self, x, y):
        
        # Check if there are any agents at the specified location
        cell_contents = self.grid.get_cell_list_contents((x, y))

        # Remove any TreeAgent(s) from the location
        for agent in cell_contents:
            if isinstance(agent, TreeAgent):
                self.grid.remove_agent(agent)
                self.schedule.remove(agent)
                
        unique_id = f"Firebreak_{x}_{y}"  # create unique ID based on coordinates
        firebreak = FirebreakAgent(unique_id, model)
        self.grid.place_agent(firebreak, (x, y))
        self.schedule.add(firebreak)
                
def manhattan_distance(point1, point2):
    return abs(point1[0] - point2[0]) + abs(point1[1] - point2[1])

# Define a function to visualize the model's state
def visualize(model):
    grid_state = {"Empty": 0, "Tree": 1, "Firefighter": 2}
    width = model.grid.width
    height = model.grid.height

    grid_matrix = np.zeros((height, width))

    for x in range(width):
        for y in range(height):
            cell_contents = model.grid.get_cell_list_contents([(x, y)])

            if any(isinstance(agent, TreeAgent) for agent in cell_contents):
                grid_matrix[y][x] = grid_state["Tree"]
            elif any(isinstance(agent, FirefighterAgent) for agent in cell_contents):
                grid_matrix[y][x] = grid_state["Firefighter"]
            else:
                grid_matrix[y][x] = grid_state["Empty"]

    plt.imshow(grid_matrix, cmap="cool", interpolation="none")
    plt.colorbar(ticks=[0, 1, 2], label='Agent Type (0: Empty, 1: Tree, 2: Firefighter)')
    plt.title("Bushfire Simulation")
    plt.show()
    


In [ ]:
import tkinter as tk  # Import Tkinter module

# Define functions for the simulation and GUI
def step_simulation():
    model.step()
    update_grid()

# Define a function to update the grid display
def update_grid():
    # Define colors for different agent types
    COLORS = {
        'T': 'green',  # Tree not on fire
        'F': 'red',    # Tree on fire
        'E': 'blue',   # Firefighter
        'B': 'gray',   # Firebreak agent
    }
    # Loop through agents to update the grid
    for agent in model.schedule.agents:
        x, y = agent.pos

        # Set the symbol based on agent type and state
        if isinstance(agent, TreeAgent):
            if agent.on_fire:
                symbol = 'F'  # Tree on fire
            else:
                symbol = 'T'  # Tree not on fire
        elif isinstance(agent, FirefighterAgent):
            symbol = 'E'  # Firefighter
        elif isinstance(agent, FirebreakAgent):
            symbol = 'B'  # FireBreak agent

        color = COLORS.get(symbol, 'white')  # Default to white if symbol not found
        # Update the label with the symbol
        grid[y][x].config(text=symbol, background=color)
        cell_width = cell_height = 20  # Adjust the size as needed

        
     # Clear the cells where firefighters have moved away
    for x in range(model.grid.width):
        for y in range(model.grid.height):
            if grid[y][x].cget("text") == 'E':
                grid[y][x].config(text=' ')

    # Restore the real firefighter's symbol as 'E'
    for firefighter in model.schedule.agents:
        if isinstance(firefighter, FirefighterAgent):
            x, y = firefighter.pos
            grid[y][x].config(text='E')

def select_fire():
    global selected_agent
    selected_agent = 'fire'
    fire_button.config(relief=tk.SUNKEN)
    firefighter_button.config(relief=tk.RAISED)
    firebreak_button.config(relief=tk.RAISED)

def select_firefighter():
    global selected_agent
    selected_agent = 'firefighter'
    firefighter_button.config(relief=tk.SUNKEN)
    fire_button.config(relief=tk.RAISED)
    firebreak_button.config(relief=tk.RAISED)
    
def select_firebreak():
    global selected_agent
    selected_agent = 'firebreak'
    firebreak_button.config(relief=tk.SUNKEN)
    fire_button.config(relief=tk.RAISED)
    firefighter_button.config(relief=tk.RAISED)
    
# Create a button to toggle the wind parameter
def toggle_wind():
    current_wind = model.wind
    if current_wind == "none":
        model.wind = "high"
        wind_button.config(text="Wind: High")
    else:
        model.wind = "none"
        wind_button.config(text="Wind: None")

# Define a function to handle cell clicks and place the selected agent
def cell_click(event):
    x = event.widget.grid_info()['column']
    y = event.widget.grid_info()['row']
    
    if selected_agent == 'fire':
        model.ignite_tree(x, y)
    elif selected_agent == 'firefighter':
        firefighter = FirefighterAgent(model.random.randint(1, 1e6), model)
        model.grid.place_agent(firefighter, (x, y))
        model.schedule.add(firefighter)
    elif selected_agent == 'firebreak':
        model.add_firebreak(x, y)  # Create a firebreak at the clicked location
        
    
    update_grid()
    # Update the cell text
    event.widget.config(text=grid[y][x].cget("text"))


# Define a function to exit the simulation cleanly
def exit_simulation():
    root.destroy()  # Close the Tkinter window and exit the simulation

# Create and run the model
width = 20
height = 20
tree_density = 1
num_firefighters = 1

model = BushfireModel(width, height, tree_density, num_firefighters, wind="none")

# Create a Tkinter window
root = tk.Tk()
root.title("Bushfire Simulation")

# Create a frame for the grid display
grid_frame = tk.Frame(root)
grid_frame.pack()

# Create a button to advance the simulation by one step
step_button = tk.Button(root, text="Step", command=step_simulation)
step_button.pack()

# Set the initial selected agent to None
selected_agent = None

# Create buttons for selecting agent type
fire_button = tk.Button(root, text="Select Fire", command=select_fire)
fire_button.pack()

firefighter_button = tk.Button(root, text="Select Firefighter", command=select_firefighter)
firefighter_button.pack()

# Create a button to select Firebreak
firebreak_button = tk.Button(root, text="Select Firebreak", command=select_firebreak)
firebreak_button.pack()

# Add a button for toggling wind in your GUI
wind_button = tk.Button(root, text="Wind: None", command=toggle_wind)
wind_button.pack()

# Create a button to cleanly exit the simulation
exit_button = tk.Button(root, text="Exit", command=exit_simulation)
exit_button.pack()

# Initialize the grid display (create cell labels with "-" and bind cell clicks)
grid = [[None for _ in range(width)] for _ in range(height)]
for x in range(width):
    for y in range(height):
        cell = tk.Label(grid_frame, text=' ', width=2, height=1)
        cell.grid(row=y, column=x)
        cell.bind('<Button-1>', cell_click)
        grid[y][x] = cell
        
update_grid()

# Start the Tkinter main loop
root.mainloop()
update_grid()


In [ ]:
# OLD TEXT BASED METHOD #
# Create and run the model
width = 5
height = 5
tree_density = 1
num_firefighters = 1

model = BushfireModel(width, height, tree_density, num_firefighters)

# Input phase
while True:  # Keep taking input until "start" is entered
    user_input = input("Enter 'f x, y' to start a fire, 'e x, y' to put out a fire, or 'start' to begin the simulation: ")
    
    if user_input.strip().lower() == "start":
        break  # Exit the input loop and start the simulation
    
    try:
        command, x, y = user_input.strip().split()
        x, y = int(x), int(y)
        if command.lower() == "f":
            # print("started fire")
            model.ignite_tree(x, y)
        elif command.lower() == "e":
            print("added firefighter")
            firefighter = FirefighterAgent(model.random.randint(1, 1e6), model)
            model.grid.place_agent(firefighter, (x, y))
            model.schedule.add(firefighter)
        else:
            print("Invalid command. Please enter 'f x, y', 'e x, y', or 'start'.")
    except ValueError:
        print("Invalid input format. Please enter commands as 'f x, y', 'e x, y', or 'start'.")

# Simulation phase
num_steps = 10  # Adjust the number of steps as needed
for i in range(num_steps):
    model.step()

    # Initialize a 2D grid to store cell states
    grid_width = model.grid.width
    grid_height = model.grid.height
    grid = [[' ' for _ in range(grid_width)] for _ in range(grid_height)]

    # Loop through agents to update the grid
    for agent in model.schedule.agents:
        x, y = agent.pos

        # Set the symbol based on agent type and state
        if isinstance(agent, TreeAgent):
            if agent.on_fire:
                grid[y][x] = 'F'  # Tree on fire
            else:
                grid[y][x] = 'T'  # Tree not on fire
        elif isinstance(agent, FirefighterAgent):
            grid[y][x] = 'E'  # Firefighter

    # Print the grid
    for row in grid:
        print(' '.join(row))
    print('-------------')